In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


2024-02-22 16:46:46.584612: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 16:46:47.230860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def keras2TFlite(model_path,name):
    #load a pre-trained model
    with tf.device('/gpu:0'):
        model = tf.keras.models.load_model(model_path)
        #convert the model
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        #save the converted model
        with open('../../model/classification/tflite/'+name+'.tflite', 'wb') as f:
            f.write(tflite_model)
        print('Model converted successfully')
        

In [3]:
keras2TFlite('../../model/classification/MobileNetV2_checkpoints.h5','MobileNetV2')
keras2TFlite('../../model/classification/MobileNetV1_acc_checkpoints.h5','MobileNetV1')
keras2TFlite('../../model/classification/DenseNet121_checkpoints.h5','DenseNet121')
keras2TFlite('../../model/classification/EfficientNetV2B0_checkpoints.h5','EfficientNetV2B0')
keras2TFlite('../../model/classification/NASNetMobile_checkpoints.h5','NASNetMobile')
keras2TFlite('../../model/classification/ResNet50_checkpoints.h5','ResNet50')


2024-02-22 16:46:51.811592: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38158 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0
2024-02-22 16:46:58.632542: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]
2024-02-22 16:47:01.046489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /tmp/tmpf2hm67r0/assets


INFO:tensorflow:Assets written to: /tmp/tmpf2hm67r0/assets
2024-02-22 16:47:16.774953: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-02-22 16:47:16.775034: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-02-22 16:47:16.776138: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpf2hm67r0
2024-02-22 16:47:16.799151: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-22 16:47:16.799178: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpf2hm67r0
2024-02-22 16:47:16.882352: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2024-02-22 16:47:16.904040: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-22 16:47:17.510413: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

Model converted successfully
